# Principios y Patrones del Desarrollo de Software.

### **Pregunta 1. SOLID.**

Explique en detalle el principio SOLID "Open/Closed" y proporcione un ejemplo de código en Python donde este principio se ha violado y cómo puede corregirlo.

Primero de todo, SOLID se trata de un acrónimo que hace referencia a los cinco principios básicos de diseño con el fin de hacer que los diseños de software sean más comprensibles.

"Open/Closed" se trata del segundo principio SOLID. Se refiere a que los módulos de software deben de ser **abiertos para su extensión, pero cerrados para su modificación**. Eso se refiere a que primero el comportamiento del software pueda extenderse. Es decir, si se cambian los requisitos, que se pueda ampliar el módulo para adaptarlo a estos. A su vez, con que el módulo este cerrado nos referimos a que si extendemos el comportamiento, los demás módulos no deben de verse afectamos por el cambio ni modificarse.

#### **Open/Closed - Modificar.**
Supongamos que tenemos una clase compra que tiene una lista de productos y sus precios. Si queremos añadir tipos de **descuentos** tendríamos que modificar la clase todo el rato.

Esto viola el principio de Open/Closed porque estamos cambiando una clase ya existente para añadir una nueva funcionalidad.

In [79]:
class Compra():
    def __init__(self):
        self.productos = []
    def add_product(self, producto, precio):
        self.productos.append([producto, precio])
    def total(self):
        total = 0
        for producto in self.productos:
            total += producto[1]
        return total
    def descuento(self, tipo_descuento):
        if tipo_descuento == "navidad":
            return self.total() * 0.80
        if tipo_descuento == "fin de mes":
            return self.total() * 0.90
        if tipo_descuento == "Black Friday":
            return self.total() * 0.50
        # y así con otros descuentos...

#### **Open/Closed - Modificado.**
Para cumplir con el principio Open/Closed se debe de aplicar el **polimorfismo**. El polimorfismo hace que diferentes objetos pueden responder de manera distinta al mismo mensaje o método. Con esto permitiríamos la agregación de distintos tipos de descuento sin necesidad de modificar la clase ya creada: `Compra`.

En lugar de modificar la clase, se creará una clase base Descuento, siendo esta abstracta ya que no implementa completamente los métodos que declara, sino que delega la implementación específica a las subclases; y luego extenderla con diferentes tipos de descuentos sin modificar la clase original.

In [80]:
from abc import ABC, abstractmethod

In [81]:
class Descuento(ABC):
    @abstractmethod
    def apply(self, compra):
        pass

class NavidadDescuento(Descuento):
    def apply(self, compra):
        return compra * 0.80

class FinDeMesDescuento(Descuento):
    def apply(self, compra):
        return compra * 0.90

class BlackFridayDescuento(Descuento):
    def apply(self, compra):
        return compra * 0.50

In [82]:
class Compra():
    def __init__(self):
        self.productos = []
    def add_product(self, producto, precio):
        self.productos.append([producto, precio])
    def total(self):
        total = 0
        for producto in self.productos:
            total += producto[1]
        return total
    def descuento(self, tipo_descuento: Descuento):
        return tipo_descuento.apply(self.total())

    def __str__(self):
        return f"Compra: {self.productos}, Total: {self.total()}"

In [83]:
if __name__ == "__main__":
    compra = Compra()
    descuentos = [BlackFridayDescuento(), NavidadDescuento(), FinDeMesDescuento()]
    compra.add_product("Camisa", 50)
    compra.add_product("Pantalón", 50)
    print(compra)
    for i in descuentos:
        print(i.__class__.__name__, ': ',compra.descuento(i))

Compra: [['Camisa', 50], ['Pantalón', 50]], Total: 100
BlackFridayDescuento :  50.0
NavidadDescuento :  80.0
FinDeMesDescuento :  90.0


In [84]:
if __name__ == "__main__":
    compra.add_product("Zapas", 100)
    print(compra)
    for i in descuentos:
        print(i.__class__.__name__, ': ',compra.descuento(i))

Compra: [['Camisa', 50], ['Pantalón', 50], ['Zapas', 100]], Total: 200
BlackFridayDescuento :  100.0
NavidadDescuento :  160.0
FinDeMesDescuento :  180.0


## **Pregunta 2. FACTORY.**
Describa el patrón de diseño "Factory". ¿En qué situaciones sería útil este patrón? Proporcione un ejemplo de cómo implementaría este patrón en Python para un problema relacionado con la ingeniería matemática, como la creación de diferentes tipos de funciones matemáticas.

**Factory** se trata de un patrón creacional. Los patrones creacionales ofrecen formas de generar instancias de clases, mejorando la adaptabilidad y la reutilización del código.

**Factory** se trata de un patrón que provee una clase abstracta, `Creator`, que permite encapsular la lógica de creación de los objetos en subclases y estas deciden que clase instanciar. Estos objetos se crean a patir de un método (NO CONSTRUCTORES). Sigue la siguiente estructura:
- Product: Definición de las interfaces para la familia de productos genéricos.
- ConcreteProduct: Implementación de los diferentes productos.
- Creator: Declara el método encargado de instanciar nuevos objetos. Suele ser una clase abstracta. Clase `Factory`.
- ConcreteCreator: Crea la instancia del producto concreto.

**Utilidades del Patrón Factory.**

El patrón Factory resulta especialmente útil cuando **no se conoce de antemano el tipo exacto de objeto a crear** y es necesario generarlo en tiempo de ejecución. Permite incorporar nuevas subclases de `Product` sin modificar el código cliente, lo que incrementa la **adaptabilidad** del sistema. Además, es ideal en escenarios donde es importante **centralizar la lógica de creación de objetos en un único punto**. Finalmente, el patrón es adecuado cuando el tipo de objeto a crear puede variar en función de determinados parámetros o condiciones, manteniendo el código limpio y flexible.


In [85]:
# CLASE Product: Función genérica que declara las operaciones que todas las funciones matemáticas tienen en común.
class MathFuncion(ABC):
    @abstractmethod
    def calcular(self, x):
        pass

In [86]:
# CLASE ConcreteProduct: Diferentes productos.
class FuncionLineal(MathFuncion):
    def __init__(self, a, b):
        self.a = a
        self.b = b
    def calcular(self, x):
        return self.a * x + self.b
    def operacion(self):
        return f'f(x) = {self.a}x + {self.b}'

class FuncionCuadratica(MathFuncion):
    def __init__(self, a, b, c):
        self.a = a
        self.b = b
        self.c = c
    def calcular(self, x):
        return self.a * x**2 + self.b * x + self.c
    def operacion(self):
        return f'f(x) = {self.a}x^2 + {self.b}x + {self.c}'

In [87]:
# CLASE Creator: Clase abstracta que declara el método de fabrica que devolverá una función.
class FuncionCreator(ABC):
    @abstractmethod
    def factory_method(self, tipo_funcion, *args) -> MathFuncion:
        pass
    def calcular(self, x, tipo_funcion, *args): # número, tipo función, coeficientes.
        funcion = self.factory_method(tipo_funcion, *args)
        return f"FuncionCreator: Calculando usando {funcion.calcular(x)} en {funcion.operacion()}"

In [88]:
# CLASE ConcreteCreator: Implementa el método de fábrica que devuelve un objeto de la clase concreta.
class ConcreteFuncionCreator(FuncionCreator):
    def factory_method(self, tipo_funcion, *args) -> MathFuncion:
        if tipo_funcion == "lineal":
            a = args[0]
            b = args[1]
            return FuncionLineal(a, b)
        if tipo_funcion == "cuadratica":
            a = args[0]
            b = args[1]
            c = args[2]
            return FuncionCuadratica(a, b, c)
        else:
            return 'Tipo de función desconocida'


In [89]:
# Código del cliente: Instancia de un creator concreto sin conocer su clase específica.
def client_code(creator: FuncionCreator, x, function_type, *args):
    print(creator.calcular(x, function_type, *args))

In [90]:
if __name__ == "__main__":
    creator = ConcreteFuncionCreator()
    
    # Ejemplo usando el creador para diferentes tipos de funciones
    print("App: Lanzado con funciones lineales.")
    client_code(creator, 5, "lineal", 2, 3)  # f(x) = 2*x + 3
    
    print("\nApp: Lanzado con funciones cuadráticas.")
    client_code(creator, 5, "cuadratica", 1, -2, 1)  # f(x) = 1*x^2 - 2*x + 1

App: Lanzado con funciones lineales.
FuncionCreator: Calculando usando 13 en f(x) = 2x + 3

App: Lanzado con funciones cuadráticas.
FuncionCreator: Calculando usando 16 en f(x) = 1x^2 + -2x + 1


## **Pregunta 3. God Object.**
Explique el antipatrón "God Object". ¿Por qué es perjudicial este antipatrón y qué problemas puede causar en el desarrollo de software? Proporcione un ejemplo de un "God Object" en un contexto de ingeniería matemática y explique cómo podría refactorizarlo para evitar este antipatrón.

**God Object** o **The Blob** es un antipatrón que asume demasiada responsabilidad, volviéndose un objeto todopoderoso que gestiona y controla la mayoría de las funciones y datos de la aplicación. El principal problema radica en que la mayoría de las responsabilidad son situadas en esta gran clase.

Este patrón es perjudicial debido a que realiza demasiadas tareas diferentes, lo que lleva a una **baja cohesión**, es decir, que sea objeto difícil de entender y mantener. Provoca una **cobertura de pruebas deficiente**, ya que al manejar múltiples responsabilidades, es complicado escribir pruebas unitarias para él. Esta estructura **dificulta el mantenimiento y actualización del software** debido a que al manejar tantas funcionalidades cualquier cambio en él puede causar efectos en cascada. La **capacidad de escalar** del sistema es **limitada**. Finalmente al manejar tantas operaciones, no puede procesar las solicitudes de manera rápida y eficiente, derivando en **problemas de rendimiento**.

### **God Object - Modificar.**
Como ejemplo a modificar creamos una clase `Estadística` que alberga distintas operaciones necesarias para el análisis de datos.

In [91]:
class Estadistica():
    def media(self, numeros: list):
        return sum(numeros) / len(numeros)

    def mediana(self, numeros: list):
        numeros.sort()
        if len(numeros) % 2 == 0:
            return (numeros[len(numeros) // 2 - 1] + numeros[len(numeros) // 2]) / 2
        else:
            return numeros[len(numeros) // 2]

    def moda(self, numeros):
        frecuencia = {}
        for num in numeros:
            if num in frecuencia:
                frecuencia[num] += 1
            else:
                frecuencia[num] = 1
        max_frecuencia = max(frecuencia.values())
        return [num for num, freq in frecuencia.items() if freq == max_frecuencia]

    def varianza(self, numeros):
        media = self.media(numeros)
        return sum((x - media) ** 2 for x in numeros) / len(numeros)

    def desviacion_estandar(self, numeros):
        return self.varianza(numeros) ** 0.5

    def rango(self, datos):
        return max(datos) - min(datos)

La clase `Estadistica`asume una gran cantidad de funcionalidades y, por lo tanto, se convierte en un componente central que gestiona diversas tareas. Aunque a primera vista puede parecer ventajoso contar con un único objeto que maneje múltiples operaciones, este enfoque frecuentemente genera complicaciones en el diseño del software:

- **Demasiadas responsabilidades.** La clase `Estadistica` maneja diversas tareas. Esto se asemeja a tener un único asistente que se encarga de múltiples tareas en una oficina.
- **Dificultad de mantenimiento.** Si quieres modificar la forma en que se calcula algún método, se necesitará entender toda la clase, lo que puede ser confuso y llevar tiempo.
- **Rendimiento lento.** Cuando se pide realizar un cálculo específico, puede que la clase tenga que recorrer todo su conjunto de funciones, lo que puede ralentizar el rendimiento, especialmente si se usan muchas operaciones al mismo tiempo.

### **God Object - Modificado.**
La solución al antipatrón consist en dividir el objeto grande en varias clases más pequeñas, cada una con su propia responsabilidad. Esto se conoce como el **principio de responsabilidad única**, SRP.

In [92]:
class MedidasCentralizadas():
    def media(self, numeros: list):
        return sum(numeros) / len(numeros)

    def mediana(self, numeros: list):
        numeros.sort()
        if len(numeros) % 2 == 0:
            return (numeros[len(numeros) // 2 - 1] + numeros[len(numeros) // 2]) / 2
        else:
            return numeros[len(numeros) // 2]

    def moda(self, numeros):
        frecuencia = {}
        for num in numeros:
            if num in frecuencia:
                frecuencia[num] += 1
            else:
                frecuencia[num] = 1
        max_frecuencia = max(frecuencia.values())
        return [num for num, freq in frecuencia.items() if freq == max_frecuencia]

class MedidasVariacion():
    def varianza(self, numeros):
        medidas_centralizadas = MedidasCentralizadas()
        media = medidas_centralizadas.media(numeros)
        return sum((x - media) ** 2 for x in numeros) / len(numeros)

    def desviacion_estandar(self, numeros):
        return self.varianza(numeros) ** 0.5

    def rango(self, datos):
        return max(datos) - min(datos)

In [93]:
if __name__ == "__main__":
    datos = [1, 2, 2, 3, 4, 4, 4, 5, 6]

    medidas_centralizadas = MedidasCentralizadas()
    print("Media:", medidas_centralizadas.media(datos))
    print("Mediana:", medidas_centralizadas.mediana(datos))
    print("Moda:", medidas_centralizadas.moda(datos))

    medidas_variacion = MedidasVariacion()
    print("Varianza:", medidas_variacion.varianza(datos))
    print("Desviación Estándar:", medidas_variacion.desviacion_estandar(datos))
    print("Rango:", medidas_variacion.rango(datos))

Media: 3.4444444444444446
Mediana: 4
Moda: [4]
Varianza: 2.246913580246914
Desviación Estándar: 1.4989708403591158
Rango: 5


## **Pregunta 4. DRY y KISS.**
Los principios DRY (Don't Repeat Yourself) y KISS (Keep It Simple, Stupid) son fundamentales para escribir código de alta calidad. Proporcione un ejemplo de un fragmento de código Python que viole estos principios. Describa cómo lo refactorizaría para adherirse a los principios DRY y KISS.

El antipatrón **Spahetti Code** se da cuándo un trozo de código no está documentado y dónde cualquier pequeño cambio o modificación hace tambalear la estructura completa del sistema. Se da cuándo se tiene una estructura de control de flujo completa e incomprensible.

Comos solución a este antipatrón es la refactorización: limpieza de código. Esto puede incluir la eliminación de código redundante y la organización de las funciones de manera más coherente. Otras soluciones consisten en aplicar los principios **DRY** y **KISS**, fundamentales para la escritura de un buen código.
- El principio **DRY** sugiere que cada pieza de conocimiento en un sistema debe tener una única representación. Es decir, evita duplicar lógica y funcionalidad.
- El principio **KISS** aboga por mantener las cosas simples y directas, evitando la complejidad innecesaria.

### **DRY y KISS - Modificar.**
Como ejemplo a modificar crearemos el cálculo del precio final de un articulo. En caso de que el precio base sea mayor  50, se aplicará el descuento y tras ello calcular el precio con impuesto. En caso contrario, únicamente se calculará el precio con impuesto sin descuento.

Finalmente retornará únicamente 100, si el valor calculado como precio final es menor-igual que 100. Sino retorna el precio final calculado.

In [94]:
def calcular_precio_final():
    precio_base = 100
    descuento = 0.1
    impuesto = 0.2

    if precio_base > 50:
        precio_base = precio_base - (precio_base * descuento)
        precio_final = precio_base + (precio_base * impuesto)
    else:
        precio_final = precio_base + (precio_base * impuesto)

    if precio_final > 100:
        return precio_final
    else:
        return 100

print(calcular_precio_final())

108.0


Sin una explicación previa, este código resulta difícil de comprender y puede llevar tiempo descifrar su funcionamiento.

- **Complejidad:** La lógica para aplicar descuentos e impuestos se encuentra entrelazada en un único bloque de código, lo que lo hace poco claro y complicado de seguir.

- **Repetición:** Si en el futuro se necesita calcular el precio de otros productos, sería necesario duplicar la misma lógica. Esto puede dar lugar a errores y complicar el mantenimiento del código.


### **DRY y KISS - Modificado.**
A continuación, se añade una versión refactorizada, más clara y sigue los principios **KISS** y **DRY**.

In [95]:
def aplicar_descuento(precio, descuento):
    return precio - (precio*descuento)

def aplicar_impuesto(precio, tasa_impuesto):
    return precio + (precio * tasa_impuesto)

def calcular_precio_final(precio_base, descuento, tasa_impuesto):
    if precio_base > 50:
        precio_base = aplicar_descuento(precio_base, descuento)

    precio_final = aplicar_impuesto(precio_base, tasa_impuesto)

    return max(precio_final, 100)

In [96]:
if __name__ == '__main__':
    precio_base = 100
    descuento = 0.1
    tasa_impuesto = 0.2
    print(calcular_precio_final(precio_base, descuento, tasa_impuesto))

108.0


## **Pregunta 5. OBSERVER.**
El patrón de diseño "Observer" permite que un objeto notifique a otros objetos sobre los cambios en su estado. Describa una situación en el contexto de la ingeniería matemática donde este patrón sería útil. Implemente un ejemplo simple de este patrón en Python para ilustrar su respuesta.

## **Pregunta Práctica 1. Refactorización de código con Principios SOLID.**
Se le proporciona un fragmento de código Python que maneja diferentes tipos de formas geométricas. Actualmente, el código viola el Principio de Responsabilidad Única (SRP) y el Principio Abierto/Cerrado (OCP) de SOLID. Su tarea es refactorizar este código para que se adhiera a estos:

### **SOLID - Modificar.**

In [102]:
class Shape:
    def __init__(self, type):
        self.type = type

class AreaCalculator:
    def __init__(self, shapes):
        self.shapes = shapes

    def total_area(self):
        total = 0
        for shape in self.shapes:
            if shape.type == "circle":
                radius = 1.0  # Supongamos que el radio es siempre 1 para este ejemplo
                total += 3.14159 * radius * radius
            elif shape.type == "square":
                side = 1.0  # Supongamos que el lado es siempre 1 para este ejemplo
                total += side * side
        return total

shapes = [Shape("circle"), Shape("square")]
calculator = AreaCalculator(shapes)
print(calculator.total_area())

4.14159


Observamos que viola los siguiente principios **SOLID**.
- **Principio de Responsabilidad Única.** La clase `AreaCalculator` es responsable tanto de mantener una lista de formas como de calcular el área de estas. Esto significa que si se quiere cambiar la forma de calcular el área (por ejemplo, añadir nuevas formas o cambiar cómo se calculan las áreas), habría que modificar la clase `AreaCalculator`, incumpliendo así el principio SOLID.

- **Open/Closed.** En el código, si se quisiera agregar nuevas formas, se tendría que modificar la clase `AreaCalculator` para incluir la lógica de cálculo del área de esas nuevas formas. Esto va en contra del principio OCP, ya que se requiere modificar el código existente en lugar de extenderlo.

### **SOLID - Modificado.**

In [97]:
class Shape(ABC):
    @abstractmethod
    def area(self):
        pass

class Circle(Shape):
    def __init__(self, radius):
        self.radius = radius
    
    def area(self):
        return 3.14159 * self.radius * self.radius

class Square(Shape):
    def __init__(self, side):
        self.side = side
    
    def area(self):
        return self.side * self.side

In [99]:
class AreaCalculator():
    def __init__(self, shapes):
        self.shapes = shapes

    def total_area(self):
        total = 0
        for shape in self.shapes:
            total += shape.area()
        return total

In [101]:
if __name__ == '__main__':
    shapes = [Circle(1.0), Square(1.0)]
    calculator = AreaCalculator(shapes)
    print(calculator.total_area())


4.14159
